In [28]:
%load_ext autoreload
%autoreload 2

In [49]:
import pandas as pd, numpy as np
import os, requests, json, warnings
import plotnine as pn

import steam

In [11]:
DATA_DIR = '../data'
data_files = sorted([os.path.join(DATA_DIR, fname) for fname in os.listdir(DATA_DIR) if fname.endswith('.csv')])

In [35]:
trn_files = data_files[:7]
sample_file = data_files[0]
reviews_dat = pd.read_csv(sample_file)

In [66]:
appid_reader = steam.AppIDReader()

In [67]:
reviews_with_names_dat = reviews_dat.merge(appid_reader.app_names_dat, on='appid', how='left')

In [68]:
reviews_with_names_dat.groupby('voted_up').size()

voted_up
False     140984
True     1319637
dtype: int64

In [69]:
review_counter = steam.ReviewCounter(reviews_with_names_dat)

In [70]:

review_counter.counts_by_game_dat

,name,reviews,total_reviews,prop
17,Counter-Strike: Global Offensive,976212,1460621,0.668354
55,Left 4 Dead 2,103661,1460621,0.070970
63,Portal 2,87843,1460621,0.060141
62,Portal,35797,1460621,0.024508
18,Counter-Strike: Source,27880,1460621,0.019088
...,...,...,...,...
50,Iron Warriors: T-72 Tank Command,118,1460621,0.000081
5,Arma: Combat Operations,115,1460621,0.000079
37,Genesis Rising,86,1460621,0.000059
93,Xpand Rally Xtreme,74,1460621,0.000051


In [73]:
review_counter.positive_negative_ratios_dat

,name,voted_up_pos,reviews_pos,total_reviews_pos,prop_pos,voted_up_neg,reviews_neg,total_reviews_neg,prop_neg,rr
9,Portal 2,True,86880,1460621,0.059482,False,963,1460621,0.000659,90.218069
13,Portal,True,35312,1460621,0.024176,False,485,1460621,0.000332,72.808247
33,Half-Life 2,True,14981,1460621,0.010257,False,317,1460621,0.000217,47.258675
37,Plants vs. Zombies: Game of the Year,True,14251,1460621,0.009757,False,367,1460621,0.000251,38.831063
5,Left 4 Dead 2,True,100694,1460621,0.068939,False,2967,1460621,0.002031,33.937984
...,...,...,...,...,...,...,...,...,...,...
265,Alpha Prime,True,226,1460621,0.000155,False,160,1460621,0.000110,1.412500
301,Earth 2160,True,156,1460621,0.000107,False,112,1460621,0.000077,1.392857
321,Safecracker: The Ultimate Puzzle Adventure,True,134,1460621,0.000092,False,108,1460621,0.000074,1.240741
110,X Rebirth,True,1206,1460621,0.000826,False,2131,1460621,0.001459,0.565931
